In [1]:
import os
# move to project root
os.chdir('/home/rcgonzal/DSC180Malware/m2v-adversarial-hindroid/')

import pandas as pd
import numpy as np

from src.model.model import M2VDroid
from src.model.hindroid import Hindroid
from src.attack.attack import *
from src.attack.cw import to_tanh_space, from_tanh_space, L2Adversary

import torch
import torch.nn as nn
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# load data 
dataset = HindroidDataset(
    'data/out/all-apps/hindroid-train-set/A_test.npz', 
    'data/out/all-apps/hindroid-train-set/predictions.csv',
    'AAT',
)

In [4]:
outfolder = 'data/out/all-apps/hindroid-train-set/'
with open(os.path.join(outfolder, 'NN_sub.pkl'), 'rb') as file:
    model = torch.load(file).to(device)

batch_size = 10
test_loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=batch_size,
    shuffle = True)

In [5]:
cw_attack = L2Adversary(
    targeted=False, confidence=0.0, c_range=(1e-1, 1e10),
    search_steps=3, max_steps=1000, abort_early=True,
    box=(0., 1.), optimizer_lr=1e-2, init_rand=True
)
advxs = None
all_inputs = []
all_advx = []

for data, target in test_loader:
    if len(all_advx) * batch_size >= 500:
        break
    
    print("OG Labels: ", target)
    target = F.softmax(model(data.to(device)), dim=1)
    print("Model pred: ", target)
    target = torch.argmax(target, dim=1)
    print(target)
    data = data.view(batch_size, -1)
    with open(os.path.join(outfolder, 'input.npz'), 'wb') as file:
        all_inputs.append(sparse.csr_matrix(data))
        sparse.save_npz(file, sparse.csr_matrix(data))
    data, target = data.to(device), target.to(device)
    advxs = cw_attack(model, data, target, to_numpy=False)
    sparse_advxs = sparse.csr_matrix(torch.round(advxs), dtype='i1')
    all_advx.append(sparse_advxs)
    break
    
#     with open(os.path.join(outfolder, 'advxs.npz'), 'wb') as file:
#         sparse.save_npz(file, sparse_advxs)
        
all_inputs = sparse.vstack(all_inputs)
all_advx = sparse.vstack(all_advx)

OG Labels:  tensor([1, 1, 0, 1, 1, 1, 1, 0, 1, 1])
Model pred:  tensor([[1.1435e-01, 8.8565e-01],
        [2.2772e-04, 9.9977e-01],
        [1.0000e+00, 2.9537e-06],
        [4.7671e-02, 9.5233e-01],
        [8.0803e-03, 9.9192e-01],
        [1.3805e-01, 8.6195e-01],
        [4.3791e-02, 9.5621e-01],
        [1.0000e+00, 4.1924e-07],
        [8.5353e-02, 9.1465e-01],
        [3.1745e-03, 9.9683e-01]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SoftmaxBackward>)
tensor([1, 1, 0, 1, 1, 1, 1, 0, 1, 1], device='cuda:0')
torch.Size([10, 2535703])
Using scale consts: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
batch [0] loss: 5.869256764300315
batch [10] loss: 38879836.945850104
batch [20] loss: 38879012.98885138
batch [30] loss: 38878901.30112338
batch [40] loss: 38878900.84094867
batch [50] loss: 38878972.6139277
batch [60] loss: 38878900.84104143
batch [70] loss: 38878900.84093383
batch [80] loss: 38878900.84100363
batch [90] loss: 38878900.84125961
batch [100] loss: 388

In [10]:
print(
    torch.min(torch.abs(advxs - 0.5)).item(),
    torch.argmax(F.softmax(model(torch.round(advxs.double().to(device))), dim=1), dim=1)
)

4.291534423828125e-06 tensor([0, 0, 1, 0, 0, 0, 0, 1, 0, 0], device='cuda:0')
